# Deploy and inference Jina Embeddings V3 with Azure app

This notebook demonstrates how to deploy a Jina Embeddings V3 model-powered [Azure managed application](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/jinaai.jina-embeddings-v3?tab=Overview) and perform inference with this application.

## Deploy the managed application

To deploy your Azure managed application, start by consulting the [official deployment guide](https://learn.microsoft.com/en-us/azure/azure-resource-manager/managed-applications/deploy-marketplace-app-quickstart). This document provides comprehensive steps for the deployment process.

It's worth mentioning that in the Basics tab of the deployment setup, you will need to provide several details about your deployment. 

You can customize the VM used, and for certain types, you might need to adjust the allowed quota to ensure access. It is recommended to use the [Standard_NC4as_T4_v3](https://learn.microsoft.com/en-us/azure/virtual-machines/nct4-v3-series) VM. This VM features up to 1 NVIDIA T4 GPU with 16 GB of memory.

<img src="images/deploy_embedding_v3_app.png" width="50%" height="50%">

Once the deployment of the managed application is complete, proceed to the resource group created for your deployment (for instance, `mrg-v3embed-20240919121306` as referenced in the provided screenshot) to verify the resources that have been established. 

Within this resource group, look for the `jina-inference-vm`. Here, you'll find the DNS Name through which you can access your application. In this example, the application is accessible via `testv2offer.eastus.cloudapp.azure.com`.

Please note that the application will be unavailable immediately after deployment due to necessary post-deployment tasks such as driver installation, dependency setup, and system reboot. **We recommend waiting at least 15 minutes before using the application.**

# Perform inference with the managed application

The Python example below demonstrates how to perform real-time inference using the DNS of the deployed Jina Embeddings V3 model-powered managed application.

In [ ]:
import json

import requests


def invoke_endpoint():
    url = "http://<Insert here your DNS prefix>.<Insert here your region>.cloudapp.azure.com:8080/encode"  # With above example, it's "http://testv2offer.eastus.cloudapp.azure.com:8080/encode"
    headers = {"Content-Type": "application/json"}
    json_data = {
        "data": [
            {
                "text": "How is the weather today?"
            },
            {
                "text": "What is the weather like today?"
            },
            {
                "text": "What's the color of an orange?"
            },
            {
                "text": "What are the hours?"
            },
            {
                "text": "Are you open on weekends?"
            },
            {
                "text": "Are you available on holidays?"
            }
        ],
        "parameters": {
            "task": "text-matching",  # Select the downstream task for which the embeddings will be used. The model will return the optimized embeddings for that task.
            "late_chunking": True,  # Apply the late chunking technique to leverage the model's long-context capabilities for generating contextual chunk embeddings.
            "dimensions": 512  # Output dimensions. Smaller dimensions are easier to store and retrieve, with minimal performance impact thanks to MRL
        } 
    }

    response = requests.post(url, headers=headers, data=json.dumps(json_data))
    print(response.json())


invoke_endpoint()